In [1]:
%load_ext autoreload
%autoreload 2

import math
import numpy as np
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint

from sklearn.feature_selection import SelectKBest
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

from model import TPALSTM
from other_models import LSTM
import pandas as pd
from dataset import ElectricityDataModule
from util import RSE, CORR
from lstnet import LSTNet

In [2]:
data_df = np.load('data/kbest_dataset.npy')
num_features = data_df.shape[1]

In [3]:
def choose_features(best_f_idx, worst_f_idx):
    data = np.load('data/dataset_norm.npy')
    X = data[:, :-1]
    y = data[:, -1]
    kbest = SelectKBest(k=worst_f_idx).fit(X, y)
    features = kbest.get_support(indices=True)[best_f_idx:]

    custom_data = np.concatenate((X[:, features], y[:, None]), axis=1)
    np.save("data/custom_dataset.npy", custom_data)
    return custom_data

custom_data = choose_features(best_f_idx=0, worst_f_idx =30)
num_features = custom_data.shape[1]

In [4]:
data_splits = {
    "train": 0.7,
    "val": 0.15,
    "predict": 0.15
}

pred_horizon = 1

elec_dm = ElectricityDataModule(
    dataset_splits=data_splits,
    batch_size=128,
    window_size=24,
    pred_horizon=pred_horizon,
    data_style="custom"
)

In [5]:
run_name = f"{pred_horizon}ts-kbest30-mlp"

In [6]:
hid_size = 64
n_layers = 1
num_filters = 3

**LSTM**

In [ ]:
wandb_logger_stacked_lstm = WandbLogger(
    name=f'{run_name}-Stacked-LSTM-{hid_size}-{n_layers}l',
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

checkpoint_loss_stacked_lstm = ModelCheckpoint(
    dirpath=f"checkpoints/{run_name}/Stacked-LSTM",
    filename='BEST-{epoch}-{val_loss:.3f}-{val_score:.3f}',
    save_top_k=1,
    monitor="val/loss",
    mode="min"
)

stacked_lstm_trainer = pl.Trainer(
    max_epochs=80,
    accelerator='gpu',
    callbacks=[checkpoint_loss_stacked_lstm],
    strategy='auto',
    devices=1,
    logger=wandb_logger_stacked_lstm
)

In [ ]:
stacked_lstm = LSTM(
    input_size=num_features,
    lstm_hid_size=hid_size,
    linear_hid_size=100,
    output_horizon=pred_horizon,
    n_layers=n_layers
)

In [ ]:
stacked_lstm_trainer.fit(stacked_lstm, elec_dm)
wandb_logger_stacked_lstm.experiment.finish()

In [ ]:
elec_dm.setup("predict")

model_path = "checkpoints/4ts-kbest/Stacked-LSTM/BEST-epoch=73-val_loss=0.000-val_score=0.000.ckpt"
stacked_lstm = LSTM.load_from_checkpoint(model_path)

pred_dl = elec_dm.predict_dataloader()

y_pred = stacked_lstm_trainer.predict(stacked_lstm, pred_dl)

In [ ]:
batch_idx = 0
for i, batch in enumerate(pred_dl):
    inputs, labels = batch
    X, ytrue = inputs[batch_idx][:, -1], labels[batch_idx].squeeze()
    ypred = y_pred[i][batch_idx].squeeze()
    
    X = X.cpu().numpy()
    ytrue = ytrue.cpu().numpy()
    ypred = ypred.cpu().numpy()
    # print(inputs.shape)
    # print(labels.shape, y_pred[i].shape)

    plt.figure(figsize=(15, 5))
    plt.plot(range(0, 24), X, label="Input")
    plt.scatter(range(24, 24 + pred_horizon), ytrue, color='red', label="True price")
    plt.scatter(range(24, 24 + pred_horizon), ypred, color='green', label="Predicted price")
    plt.legend()
    plt.title("Stacked-LSTM")
    plt.show()

    if i == 3:
        break

**TPA-LSTM**

In [ ]:
# name = f'{run_name}-TPA-LSTM-{hid_size}-{num_filters}f-{n_layers}l'
name = f'{run_name}-TPA-LSTM'

wandb_logger_tpalstm = WandbLogger(
    name=name,
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

checkpoint_loss_tpalstm = ModelCheckpoint(
    dirpath=f"checkpoints/{run_name}/TPA-LSTM",
    filename=name,
    save_top_k=1,
    monitor="val/loss",
    mode="min"
)

tpalstm_trainer = pl.Trainer(
    max_epochs=60,
    # accelerator='gpu',
    callbacks=[checkpoint_loss_tpalstm],
    strategy='auto',
    devices=1,
    logger=wandb_logger_tpalstm
)

In [ ]:
tpa_lstm = TPALSTM(
    input_size=num_features,
    hidden_size=hid_size,
    output_horizon=pred_horizon,
    num_filters=num_filters,
    obs_len=24,
    n_layers=n_layers,
    lr=1e-3
)

In [ ]:
tpalstm_trainer.fit(tpa_lstm, elec_dm)
wandb_logger_tpalstm.experiment.finish()

In [ ]:
elec_dm.setup("predict")

model_path = f"checkpoints/{run_name}/TPA-LSTM/BEST-epoch=35-val_loss=0.000-val_score=0.000.ckpt"
tpa_lstm = TPALSTM.load_from_checkpoint(model_path)

pred_dl = elec_dm.predict_dataloader()

y_pred = tpalstm_trainer.predict(tpa_lstm, pred_dl)

In [ ]:
batch_idx = 0
for i, batch in enumerate(pred_dl):
    inputs, labels = batch
    X, ytrue = inputs[batch_idx][:, -1], labels[batch_idx].squeeze()
    ypred = y_pred[i][batch_idx].squeeze()
    
    X = X.cpu().numpy()
    ytrue = ytrue.cpu().numpy()
    ypred = ypred.cpu().numpy()
    # print(inputs.shape)
    # print(labels.shape, y_pred[i].shape)

    plt.figure(figsize=(15, 5))
    plt.plot(range(0, 24), X, label="Input")
    plt.scatter(range(24, 24 + pred_horizon), ytrue, color='red', label="True price")
    plt.scatter(range(24, 24 + pred_horizon), ypred, color='green', label="Predicted price")
    plt.legend()
    plt.title("TPA-LSTM")
    plt.show()

    if i == 3:
        break

**LSTNet**

In [7]:
name = f'{run_name}-LSTNet'

wandb_logger_lstnet = WandbLogger(
    name=name,
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

checkpoint_loss_lstnet = ModelCheckpoint(
    dirpath=f"checkpoints/{run_name}/LSTNet",
    filename=name,
    save_top_k=1,
    monitor="val/loss",
    mode="min"
)

lstnet_trainer = pl.Trainer(
    max_epochs=60,
    # accelerator='gpu',
    callbacks=[checkpoint_loss_lstnet],
    strategy='auto',
    devices=1,
    logger=wandb_logger_lstnet
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fraikinarchie (mva-data-challenge). Use `wandb login --relogin` to force relogin


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
lstnet = LSTNet(
    num_features=31,
    window_size=24,
    conv1_out_channels=32, 
    conv1_kernel_height=6,
    recc1_out_channels=64, 
    skip=3, 
    skip_reccs_out_channels=6, 
    hw_window_size=7,
    output_fun="sigmoid"
)

In [9]:
lstnet_trainer.fit(lstnet, elec_dm)
wandb_logger_lstnet.experiment.finish()

/Users/damso/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /Users/damso/MVA/TimeSeries/tpa-lstm-pytorch/checkpoints/1ts-kbest30-mlp/LSTNet exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name      | Type    | Params
--------------------------------------
0 | conv1     | Conv2d  | 6.0 K 
1 | GRU1      | GRU     | 18.8 K
2 | dropout   | Dropout | 0     
3 | GRUskip   | GRU     | 720   
4 | linear1   | Linear  | 2.6 K 
5 | highway   | Linear  | 8     
6 | criterion | MSELoss | 0     
--------------------------------------
28.1 K    Trainable params
0         Non-trainable params
28.1 K    Total params
0.112     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/damso/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/damso/opt/anaconda3/envs/venv/lib/python3.8/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/damso/opt/anaconda3/envs/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([128, 1])) that is different to the input size (torch.Size([128, 31])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_los

Training: 0it [00:00, ?it/s]

/Users/damso/opt/anaconda3/envs/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([71, 1])) that is different to the input size (torch.Size([71, 31])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/damso/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightning/pytorch/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
train/corr,▁▄▅▆▇▇▇█████
train/loss,█▆▄▃▃▂▂▂▂▁▁▁
train/rse,█▆▄▄▃▂▂▂▂▁▁▁
trainer/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
val/corr,▁▄▆▆▆▇▇▇████
val/loss,█▇▆▅▄▄▅▅▄▃▁▁
val/rse,███▇▅▆▇▇▆▄▁▁
epoch,11
train/corr,3.77245
train/loss,0.00803


**VAR**

In [ ]:
means = custom_data.mean(axis=0)
stds = custom_data.std(axis=0)

normed_data = (custom_data - means) / stds

In [ ]:
train_lim = math.floor(data_splits['train'] * custom_data.shape[0]) 
test_lim =  math.floor(data_splits['val'] * custom_data.shape[0]) + train_lim

train_data = normed_data[:train_lim]
test_data = normed_data[test_lim:]

In [ ]:
for i in range(train_data.shape[1]):
    plt.figure(figsize=(15, 4))
    plt.plot(train_data[:, i])
    plt.show()

In [ ]:
var_model = VAR(train_data)
var_res = var_model.fit(24)


In [ ]:
n = test_data.shape[0] - (24 + pred_horizon)
ypreds = np.zeros((n, pred_horizon))
ytrue = np.zeros((n, pred_horizon))

for i in range(n):
    ypreds[i] = var_res.forecast(test_data[i: i +24], 4)[:, -1]
    ytrue[i] = test_data[i + 24: i + 24 + pred_horizon, -1]
    
var_rse = RSE(ypreds, ytrue)
var_corr = CORR(ypreds, ytrue)
print(f"VAR predictions - RSE: {var_rse}, CORR: {var_corr}")